# Decision Tree Learning

When it comes to the problem of weak AI i.e. agents that act intelligent but are not conscious, there are many approaches to acheiving the goal of intellligence depending on the task. Some tasks such as (some) games "only" need clever algorithms at run-time as to acheive this. The algorithms may be very rudimentary in some cases. For example in tic-tac-toe, we wouldn't even have to use clever algorithms like minimax or alpha-beta search; one could easily hard code the best move for each state. Search algorithms are paricularly efficient for a wide range of applications,given good heuristic functions. Nevertheless, some cases call for another tool entirely, such as planning or learning. Today, we will take a look at the latter.


Learning is the process of an agent improving its future performance by adapting its output based on a given percept sequence and knowledge about the world. Learning is an important tool as the perfect behavior for an agent may not be programmable. This can be due to a dynamic environment that prevents a programmer from implementing every possible action, e.g., the environment of a humanoid robot or the Mars exploration rovers, which can take steps of varying lengths. Or it may be due to a dynamic environment in which not every outcome is predictable, e.g., the stock market. Three forms of learning are common in artificial intelligence: supervised learning, unsupervised learning, and reinforcement learning. These types of learning differ in the form of feedback available for the agent. In supervised learning, both the input and output are available to the agent; reinforcement learning does not provide the output to the agent but provides feedback in the form of a reward or a punishment, and unsupervised learning does not provide any feedback. Decision Tree Learning is an inductive (supervised) form of learning where the agent learns a function based on given input-output pairs.

## Motivation

I was introduced to decision tree learning as part of the "Intro to AI" course in the previous semester and had seen and executed the basic algorithm by hand. I was interested in learning how it was implemented in popular modules such as scikit-learn and gaining insight into some higher-level concepts for decision trees that I hadn't been introduced to yet.

I completed a course on [Kaggle](https://www.kaggle.com/), "Intro to Machine Learning," in which I was introduced to the DecisionTreeRegressor class and learned some basics of model evaluation based on data from the Iowa housing market. My main goal for this project was to improve my proficiency in pandas, matplotlib, scikit-learn, and practice using the core functions. I wanted to do some plotting and modeling and gain insight from it.

As I continue learning, I may update this project or add other notebooks to this repository.


## Dependencies

This notebook uses the following dependencies. Please ensure you have Python installed. I used VS Code when writing down the execution instructions, so if you are not familiar with venvs, kindly download it; it provides a straightforward way to create them without having to use the command line. I will not rehash how to create a venv here. If you need a guide on that, please check the subsections "Setting up a venv", "What is a venv", and "Creating a venv" [here](https://github.com/Kena-Njonge/Grade_Analysis)

The dependencies will be installed when you create the venv.

- [Python](https://www.python.org/) (version 3.12.2 or later)
- [Pandas](https://pandas.pydata.org/) (version 2.2.1 or later)
- [Matplotlib](https://matplotlib.org/) (version 3.8.0 or later)
- [scikit-learn](https://scikit-learn.org/stable/) (version 1.3.0 or later)
- [Ipython Kernel](https://ipython.org/) (version 6.29.3 or later)
- [Numpy](https://numpy.org/) (version 1.26.4 or later)




## Decion Trees.

Decision Trees are a simple model, one of their main advantages is their explainability, as a user can clearly understand what steps were taken and what considerations there were in the generation of a certain output. They are also the basic building blocks for some more complicated models in data science, e.g., random forests. Decision Trees' main weakness is they are prone to overfitting; deep decision trees will capture noise and memorize data.


There are two types of decision trees: classifiers and regression-based decision trees. The differentiator between the two is if the value that we want to predict, our target, and our features are all discrete or if some are continuous. If the values are discrete, we can use classification; otherwise, we use regression. Apart from this, the methods for creating a model are pretty similar. The model is trained by splitting the training data based on some attribute. We choose the attribute that either maximizes or minimizes a target value, depending on the context. This selection could be based on criteria such as the Gini coefficient, Information Gain, or Mean Squared Error. Despite their limitations, decision trees have shown themselves to be very useful in the industry.

## The Dataset

The dataset that I will use in this notebook was provided by Anaconda as part of their "Data Analysis with Excel and Python course," which I completed. The course itself was mostly focused on showing that core Excel functionality such as pivot tables can just as easily be implemented in Python. For this, we used Pandas.

The dataset consists of 392 entries with 8 attributes. Each entry consists of data about a car, e.g., its name, how many miles per gallon it achieves, the year it was built, and its origin. The dataset was provided with no NaN rows or other peculiarities, so there was no preprocessing needed.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np 
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn import tree

file_path = r".\auto_data.xlsx"
df = pd.read_excel(file_path)

df.shape


In [ ]:
attribute_list = list(df.columns)
print(attribute_list)
#The data has two unneeded columns, we delete these.
df = df[attribute_list[0:9]]
df['Year'] = df['yr'] + 1900 
df.head()

In [ ]:
#It never hurts to run a df.describe()
df.describe()
#Notice that non-numeric columns will (obviously) not be included. 

In [ ]:
#Let's plot the relationship between mpg (miles per gallon) and the Year
fig,ax = plt.subplots()
X = df['Year']
y = df['mpg']
ax.scatter(X,y)
ax.set_ylabel("mpg")
ax.set_xlabel("Year")
plt.show()

Visually, one can observe a general upward trend in miles per gallon (mpg) as the years increase, indicating that newer cars tend to be more energy efficient. Additionally, there is noticeable variance in mpg across different years. To analyze this further, let's group our values into bins and quantify the spread per year. Instead of using prebuilt modules, we'll take a naive approach. But before we do that, let's formalize and quantify the correlation that we observe and determine the regression line, also known as the line of best fit.


The correlation coefficient used in numpy is the Pearson correlation coefficient. Numpy utilitzes least square method to determine the regression line. If you want more information on how to determine the regression line, or if you're interested in implementing it yourself without external libraries, you can refer to [this article](https://www.varsitytutors.com/hotmath/hotmath_help/topics/line-of-best-fit) by Varsity Tutors where they go over the calculation in depth. 


In [ ]:
fig,ax = plt.subplots()
X = df['Year']
y = df['mpg']
ax.scatter(X,y)

corr_coeff = np.corrcoef(X,y)

#The parameter 1 specifies the degree of the polynomial to fit.
slope, intercept = np.polyfit(X,y,1)

regression_line = slope * X + intercept
plt.plot(X, regression_line, color='red', label='Regression_Line')

plt.title('Scatter Plot with Regression Line')
plt.xlabel('year')
plt.ylabel('mpg')
plt.show()
print(f"Our correlation coefficient is {corr_coeff[0][1]} . We have a positive correlation between 'mpg' and 'year'.")

Using gradient descent can potentially yield an even better regression line. Gradient descent is a fundamental algorithm in machine learning. If you're interested in learning more about the algorithm and understanding it from a mathematical standpoint, [this medium article](https://medium.com/analytics-vidhya/linear-regression-with-gradient-descent-derivation-c10685ddf0f4) helped me understand it better.

Although I won't be covering gradient descent in this notebook, I may do so at a later date. What I will demonstrate next is that you can achieve the same regression line using the LinearRegression class in sklearn. In my last notebook, I didn't plot the regression line; instead, I plotted the predictions. While this approach isn't necessarily incorrect, it might have caused some confusion, as readers generally expect to see a line when the author discusses linear regression.

In [ ]:
lin_reg = LinearRegression()

# Creating the test and train sets and creating our model, 
# We did not plot in the predictions this time.
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
X_train = X_train.values.reshape(-1,1)
lin_reg.fit(X.values.reshape(-1,1), y)
y_pred_lin = lin_reg.predict(X_test.values.reshape(-1,1))

fig,ax = plt.subplots()
ax.scatter(X,y)

# Plotting the regression line
slope_2 = lin_reg.coef_
intercept_2 = lin_reg.intercept_
regression_line_2 = intercept_2 + slope_2 * X

plt.plot(X, regression_line_2, color='blue', label='Regression_Line ')


plt.title('Scatter Plot with Regression Line')
plt.xlabel('year')
plt.ylabel('mpg')
plt.show()

One can observe a significant amount of variance in the miles per gallon achieved by cars produced in different years.

In [ ]:
#We can use series.uniques as to quickly count the number of years
unique_years = df['Year'].unique()

fig1, axes = plt.subplots(1,len(unique_years), figsize=(20,5))
#If you want to do something with the index of a list element, or iterable but also need to iterate over the elements themselves 
# use the enumerate function
standard_deviation = dict()
for year_index, year in enumerate(unique_years):
        year_df = df[df['Year']==year]
        standard_deviation[year] = year_df.describe().loc['std','mpg']
        axes[year_index].boxplot(year_df['mpg'])
        #Use an f-string a (formatted literal string) to quickly deal with the multiple titles
        axes[year_index].set_title(f"mpg in {year}")
plt.show()
print("The standard deviations are as follows.")
print(standard_deviation)

In [ ]:
max_deviation = max(standard_deviation.values())
min_deviation = min(standard_deviation.values())

#It is smarter to iterate over items, the (key,value) pair, as I need to access both the key and the value 
for key, value in standard_deviation.items():
    if value==max_deviation:
        print(f"The maximum standard deviation was {max_deviation} in the year {key}")
    if value==min_deviation:
        print(f"The minimum standard deviation was {min_deviation} in the year {key}")
    

We can observe that the year with the largest variance in mpg is 1978. This variance, which represents the spread or dispersion of the mpg values around the mean, is calculated as the square of the standard deviation. So it follows from our calculation of the standard deviation.

In [ ]:
sorted_deviations = sorted(standard_deviation.items(), key=lambda item: item[1])

fig, ax = plt.subplots()

# I will not lie, I still do not understand why I constantly have to reshape some arrays

X_year_var = np.array([sub_array[0] for sub_array in sorted_deviations]).reshape(-1, 1)
y_year_var = np.array([sub_array[1] for sub_array in sorted_deviations]).reshape(-1, 1)

ax.scatter(X_year_var, y_year_var)
ax.set_xlabel("year")
ax.set_ylabel("standard deviation of mpg")

var_reg = LinearRegression()
var_reg.fit(X_year_var, y_year_var)

var_intercept = var_reg.intercept_
var_slope = var_reg.coef_

#Just as I thought for some reason this method does not work if I reshape the data haha
#Plotting the regression line. 
corr_coef_var = np.corrcoef([sub_array[0] for sub_array in sorted_deviations],[sub_array[1] for sub_array in sorted_deviations])
var_reg_func = var_intercept + var_slope * X_year_var
ax.plot(X_year_var, var_reg_func, color='red', label="Regression Line")
plt.show()

print(f"Our correlation coefficient is {corr_coef_var[0][1]} . We have a positive correlation between 'standard deviation of mpg' and 'year'.")


## Creating the Model

Now that we have a solid understanding of what the dataset looks like, along with some correlations and characteristics, we can proceed with creating a model. As discussed in my previous notebook, to build an effective model, we need to evaluate its performance. For this, we must split the data into training and test subsets. Scikit-learn provides a function for this in the `model_selection` submodule, typically splitting the data in a 3:1 ratio by default.

Before proceeding, we need to define our features (the attributes we use for prediction) and our prediction target. There's generally no fixed rule for determining the optimal number of features for a given target variable.

We'll designate our target variable as "mpg" and consider the other columns as our features. In data analysis, two types of decision trees are commonly used: classification trees and regression trees. Regression tree analysis is employed when the predicted outcome can be considered a real number. As expanded upon in Russel and Norvig's book "Artificial Intelligence: A Modern Approach": "A regression tree has at each leaf a linear function of some subset of numerical attributes, rather than a single value. For example, the branch for two-bedroom apartments might end with a linear function of square footage, number of bathrooms, and average income for the neighborhood." Given that we are dealing with continuous values, we'll opt for a regression tree.

Before proceeding with model creation, we need to transform non-numeric items, such as car names and their origins, into numeric equivalents. Creating a mapping from origin to a number is straightforward since we only have three values, allowing us to hard-code the mapping. However, mapping names to numbers is a bit more involved. We'll store the mapped values in `origin_id` and `name_id` respectively.


In [ ]:
df['origin'].unique()

In [ ]:
names = list(df['name'])

name_dict = defaultdict(list)

#We need to classify all the names, my approach is first to populate a dict with the different names,
#then transform it into a list and have our mapping be based on the list index
#I now know a more efficient way to implement this, but I leave that task to you 
#Hint: You can just use the .unique() method
for index, name in enumerate(names):
    name_dict[name].append(index)

#Create new columns
df['name_id'] = 0
df['origin_id'] = 0


fetch_first_elements = lambda arr: [subarr[0] for subarr in arr]

#We used this to get all the car names after classifying them in the first step. 
#This is part of the suboptimal initial approach
name_list = fetch_first_elements(list(name_dict.items()))


#This is the mapping portion
for index in df.index:
    df.loc[index,'name_id'] = name_list.index(df.loc[index,'name'])
    if df.loc[index,'origin'] == 'US':
        df.loc[index,'origin_id'] = 0
    elif df.loc[index,'origin'] == 'Asia':
        df.loc[index,'origin_id'] = 1
    else:
        #For Europe. All entries in our df have an origin, so this is possible
         df.loc[index,'origin_id'] = 2

df.head()

We do a final data formatting step. Dropping the original 'yr' column as we do it need it anymmore.


In [ ]:
try:
    df = df.drop("yr", axis=1)
except:
    pass
df.head()

We are now ready to create the model. We split the data into training and test sets and then train multiple decision trees with different depths. Next, we evaluate their performance using the test data based on the mean absolute error of each one. We choose the best-performing model and display it. Additionally, the decision tree will be saved in the current folder, allowing for better viewing of its nodes.

In [ ]:
X_1 = df[['cyl', 'displ', 'hp', 'weight', 'accel', 'origin_id', 'name_id',
       'Year']]

X_train, X_test, y_train, y_test = train_test_split(X_1,y,random_state=42)

# We select a couple of depths to test.
# As far as I understand, it doesn't make any sense to have a higher depth than we have features
depths = [4, 5, 6, 7, 8]

# Training decision trees with different depths and evaluating performance
mean_abs_errors = []
for depth in depths:
    mpg_model = DecisionTreeRegressor(max_depth=depth, random_state=42)
    mpg_model.fit(X_train, y_train)
    y_prediction = mpg_model.predict(X_test)
    mean_abs_errors.append(mean_absolute_error(y_test, y_prediction))

# Find the best depth based on mean absolute errors
best_depth = depths[mean_abs_errors.index(min(mean_abs_errors))]

print(f"The decision tree performs best with a depth of {best_depth}")
print(f"The mean absolute errors of the different depths are: {mean_abs_errors}")

# Plotting the decision tree
fig, ax = plt.subplots(figsize=(70, 50))
tree.plot_tree(mpg_model, ax=ax, filled=True, feature_names=list(X_1.columns))  

# Saving the plot if it doesn't already exist
if not os.path.exists("Decision_Tree.png"):
    plt.savefig("Decision_Tree.png")
    print("Decision tree visualization saved as 'Decision_Tree.png'")
else:
    print("File 'Decision_Tree.png' already exists")

plt.show()


## Final Outlook and areas for improvement.

Overall, I am quite satisfied with what I was able to create. I had a lot of fun going through this modeling process and learning new modules, as well as sharpening my skills with known ones. In hindsight, I think similar model quality could have been achieved with fewer features. To achieve this, I should have first examined if there is a correlation between the other variables and mpg. If the correlation is minor compared to the others, then it would be fair to assume that these do not bring us much more information when our goal is calculating the mpg. I am also a little disappointed in the depth of the tree; we are bordering on overfitting as we have 8 possible features, and we use 7 in our decision tree. The biggest improvement that I think I could make on my next go is using a random forest as my model.